# Food.com Reviews data analysis

In [72]:
#Importing Necessary packages
import sqlite3
import numpy as np
import pandas as pd

In [73]:
#Establishing Connecting to the SQLite Database and listing the table names

con = sqlite3.connect(r"C:\Users\Admin\Food\database.sqlite")
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Reviews',)]


In [76]:
#Reading SQL Table content using Pandas Function.

df = pd.read_sql_query('select * from Reviews',con)

In [135]:
#Exploring the column details

df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [81]:
df['Text'][3]

'If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal.'

In [82]:
#Function to categorize score ranges 0-5 as Positive and Negative

def score_categorize(x):
    if x < 3:
        return 'Negative'
    else:
        return 'Positive'
    

In [84]:
#Renaming the column content to category

df['Score'] = df['Score'].map(score_categorize)

In [86]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,Positive,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,Negative,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,Positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,Positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


# Data Cleaning

In [118]:
#Filtering duplicate records using SQL query based on User ID and Time 

pd.read_sql_query('select UserId, Time, count(*) \
                  from Reviews \
                  group by UserId,Time \
                  having count(*) > 1 \
                  order by count(*) desc \
                  ',con)

,UserId,Time,count(*)
0,A3TVZM3ZIXG8YW,1291420800,199
1,A29JUMRL1US6YP,1278201600,125
2,AJD41FBJD9010,1233360000,73
3,ABDCYK04CL6O4,1323993600,68
4,A26NFIQ7KWI8Y7,1329696000,65
...,...,...,...
76852,AZZA4Q0JACD5U,1340582400,2
76853,AZZH3GGYQSBUC,1229299200,2
76854,AZZTH6DJ0KSIP,1304208000,2
76855,AZZU4D6TZ2L6J,1247875200,2


In [122]:
#Exploring Duplicate records based on Individual user id with review at same time for a product

df[(df['UserId'] == 'A3TVZM3ZIXG8YW') & (df['Time'] == 1291420800)]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2941,2942,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,7,11,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
2947,2948,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,0,2,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
31782,31783,B00106TG9Y,A3TVZM3ZIXG8YW,christopher hayes,2,8,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
52496,52497,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,19,21,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
52501,52502,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,18,24,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
...,...,...,...,...,...,...,...,...,...,...
499916,499917,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,14,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
499917,499918,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,15,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
514140,514141,B003M5VM8O,A3TVZM3ZIXG8YW,christopher hayes,5,9,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."
514690,514691,B003MWGSKY,A3TVZM3ZIXG8YW,christopher hayes,3,8,Negative,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ..."


In [124]:
#Arranging Dataframe in sorted manner based on Product ID

df.sort_values(by = 'ProductId',axis=0,ascending= True)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,Positive,1108425600,"Nice cadence, catchy rhymes",In June<br />I saw a charming group<br />of ro...
150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
150504,150505,0006641040,A2PTSM496CF40Z,"Jason A. Teeple ""Nobody made a greater mistak...",1,1,Positive,1210809600,A classic,Get the movie or sound track and sing along wi...
150503,150504,0006641040,AQEYF1AXARWJZ,"Les Sinclair ""book maven""",1,1,Positive,1212278400,Chicken Soup with Rice,A very entertaining rhyming story--cleaver and...
...,...,...,...,...,...,...,...,...,...,...
191720,191721,B009UOFTUI,AJVB004EB0MVK,D. Christofferson,0,0,Negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
1477,1478,B009UOFU20,AJVB004EB0MVK,D. Christofferson,0,0,Negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
328481,328482,B009UUS05I,ARL20DSHGVM1Y,Jamie,0,0,Positive,1331856000,Perfect,The basket was the perfect sympathy gift when ...
5702,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [137]:
#Dropping duplicate entries from Dataframe based on User ID , Text and Time 

df_cor = df.drop_duplicates(subset= ['UserId','Time','Text'],inplace= False)

In [128]:
df_cor.shape

(393892, 10)

In [132]:
df_cor

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,Positive,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,Negative,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,Positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,Positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [138]:
#Percentage of Data after Duplicate removed
df_cor['Id'].size / df['Id'].size * 100

69.29179845686723